# Проект по SQL

**Цель исследования:** Проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных:**

**Таблица `books` cодержит данные о книгах:**
- *book_id* — идентификатор книги;
- *author_id* — идентификатор автора;
- *title* — название книги;
- *num_pages* — количество страниц;
- *publication_date* — дата публикации книги;
- *publisher_id* — идентификатор издателя.

**Таблица `authors` cодержит данные об авторах:**
- *author_id* — идентификатор автора;
- *author* — имя автора.

**Таблица `publishers` cодержит данные об издательствах:**
- *publisher_id* — идентификатор издательства;
- *publisher* — название издательства.

**Таблица `ratings` cодержит данные о пользовательских оценках книг:**
- *rating_id* — идентификатор оценки;
- *book_id* — идентификатор книги;
- *username* — имя пользователя, оставившего оценку;
- *rating* — оценка книги.

**Таблица `reviews` cодержит данные о пользовательских обзорах:**
- *review_id* — идентификатор обзора;
- *book_id* — идентификатор книги;
- *username* — имя автора обзора;
- *text* — текст обзора.

**Задания:**  
  
`Задача 1.` Посчитайте, сколько книг вышло после 1 января 2000 года  
`Задача 2.` Для каждой книги посчитайте количество обзоров и среднюю оценку  
`Задача 3.` Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры  
`Задача 4.` Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками  
`Задача 5.` Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок  

**План проекта:**  
  
`Шаг 1.` Исследование таблиц (вывод первых строк)  
`Шаг 2.` Решение задач и вывод результатов  
`Шаг 3.` Выводы по задачам  

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# функция для вывода результата
def result(query):
    con=engine.connect()
    return pd.io.sql.read_sql(sql=text(query), con = con)

## Исследование таблиц

### Таблица `books`

In [3]:
query = '''
        SELECT *
        FROM books
        LIMIT 1;
        '''
result(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


### Таблица `authors`

In [4]:
query = '''
        SELECT *
        FROM authors
        LIMIT 1;
        '''
result(query)

,author_id,author
0,1,A.S. Byatt


### Таблица `publishers`

In [5]:
query = '''
        SELECT *
        FROM publishers
        LIMIT 1;
        '''
result(query)

,publisher_id,publisher
0,1,Ace


### Таблица `ratings`

In [6]:
query = '''
        SELECT *
        FROM ratings
        LIMIT 1;
        '''
result(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


### Таблица `reviews`

In [7]:
query = '''
        SELECT *
        FROM reviews
        LIMIT 1;
        '''
result(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


## Решение задач

### Задача 1. Посчитайте, сколько книг вышло после 1 января 2000 года.

In [8]:
query = '''
        SELECT COUNT(book_id) AS count_books
        FROM books
        WHERE publication_date > '2000-01-01';
        '''
result(query)

,count_books
0,819


### Задача 2. Для каждой книги посчитайте количество обзоров и среднюю оценку.

In [9]:
query = '''
        WITH
        cu AS (SELECT book_id,
                      COUNT(review_id) as count_reviews
               FROM reviews
               GROUP BY book_id),
        rat AS (SELECT book_id,
                       AVG(rating) as average_rating
                FROM ratings
                GROUP BY book_id)
        SELECT b.title,
               cu.count_reviews,
               ROUND(rat.average_rating, 2) AS average_rating
        FROM books AS b
        JOIN cu ON cu.book_id = b.book_id
        JOIN rat ON rat.book_id = b.book_id
        GROUP BY b.title,
                 cu.count_reviews,
                 average_rating;
        '''
result(query)

,title,count_reviews,average_rating
0,Cane River,2,4.50
1,The Summons,3,4.33
2,The Mad Ship (Liveship Traders #2),2,4.50
3,Chapterhouse: Dune (Dune Chronicles #6),2,4.00
4,Ella Enchanted (Ella Enchanted #1),4,4.08
...,...,...,...
989,Swann's Way (In Search of Lost Time #1),2,3.00
990,The 6th Target (Women's Murder Club #6),2,3.50
991,Rachel's Holiday (Walsh Family #2),2,3.50
992,John Adams,4,3.80


### Задача 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

In [10]:
query = '''
        SELECT publ.publisher,
               COUNT (p.book_id) AS count_books
        FROM (SELECT book_id,
                publisher_id
                FROM books
                WHERE num_pages > 50) AS p
        JOIN publishers AS publ ON publ.publisher_id = p.publisher_id
        GROUP BY publisher
        ORDER BY count_books DESC
        LIMIT 1;
        '''
result(query)

,publisher,count_books
0,Penguin Books,42


### Задача 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

In [11]:
query = '''
        SELECT
        a.author,
        ROUND(AVG(rat.average_rating), 2) AS average_rating    
        FROM books AS b
        LEFT JOIN (SELECT book_id,
                          COUNT(rating) AS count_rat,
                          AVG(rating) AS average_rating
                   FROM ratings
                   GROUP BY book_id) AS rat ON rat.book_id = b.book_id
        LEFT JOIN authors AS a ON a.author_id = b.author_id
        WHERE rat.count_rat >= 50
        GROUP BY author
        ORDER BY average_rating DESC
        LIMIT 1;   
        '''
result(query)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.28


### Задача 5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [12]:
query = '''
        WITH
        rat AS (SELECT username  
                FROM ratings
                GROUP BY username
                HAVING COUNT(rating_id) > 48),
        rev AS (SELECT username,
                COUNT(review_id) AS count_review
                FROM reviews
                GROUP BY username)
        SELECT AVG(rev.count_review) AS average_reviews
        FROM rat
        LEFT JOIN rev ON rev.username = rat.username;
        '''
result(query)

,average_reviews
0,24.0


## Вывод по задачам

- После 1 января 2000 года вышло **819 книг**.
- Для каждой книги посчитали количество обзоров и среднюю оценку, книг в списке 994.
- Издательство, которое выпустило наибольшее число книг толще 50 страниц - **Penguin Books**, 42 книги.
- Автор с самой высокой средней оценкой книг (учитывали только книги с 50 и более оценками) - **J.K. Rowling/Mary GrandPré** с оценкой 4.28.
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - **24**.